In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('agg')
from matplotlib.figure import Figure
import panel as pn
import statistics
import math
import param
import io
import os
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from patsy import dmatrices
from panel.viewable import Viewer
from scipy import stats
import base64

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
data_input = pd.read_excel('/Users/ctlewis/Documents/Projects/Chaxas/zircon_data/zircon_10182022/zirconone_lasertramzii_rdy/zirconone_six.xls',
                    sheet_name='Buffer')
data_input['Time_s'] = data_input.Time/1000
options = list(data_input.SampleLabel.unique())

In [4]:
# function block #

def get_ratios(data):
    og_len = len(data.columns)
    data = data.reset_index(drop=True)
    data_ratio = data.copy()
    pb206_pb204 = []
    pb206_u238 = []
    pb207_u235 = []
    pb207_pb206 = []
    u235_u238 = []
    pb206_u238c = []
    
    data_ratio['238c'] = data_ratio['235U'] * 137.82
    data_ratio['Time_s2'] = data_ratio['Time_s']**2
    
    for i in range(0,len(data_ratio)):
        if data_ratio['238U'][i] > 0:
            pb206_u238.append((data_ratio['206Pb'][i]/data_ratio['238U'][i]))
        else:
            pb206_u238.append(0)
    
    for i in range(0,len(data_ratio)):
        if data_ratio['204Pb'][i] > 0:
            pb206_pb204.append((data_ratio['206Pb'][i]/data_ratio['204Pb'][i]))
        else:
            pb206_pb204.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['235U'][i] > 0:
            pb207_u235.append((data_ratio['207Pb'][i]/data_ratio['235U'][i]))
        else:
            pb207_u235.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['206Pb'][i] > 0:
            pb207_pb206.append((data_ratio['207Pb'][i]/data_ratio['206Pb'][i]))
        else:
            pb207_pb206.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['238U'][i] > 0:
            u235_u238.append((data_ratio['235U'][i]/data_ratio['238U'][i]))
        else:
            u235_u238.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['238c'][i] > 0:
            pb206_u238c.append((data_ratio['206Pb'][i]/data_ratio['238c'][i]))
        else:
            pb206_u238c.append(0)
        
    data_ratio['Pb206_Pb204'] = pb206_pb204
    data_ratio['Pb206_U238'] = pb206_u238
    data_ratio['Pb207_U235'] = pb207_u235
    data_ratio['Pb207_Pb206'] = pb207_pb206
    data_ratio['U235_U238'] = u235_u238
    data_ratio['Pb206_U238c'] = pb206_u238c
    
    data_ratio = data_ratio.iloc[:,(og_len-1):]
    
    return data_ratio



def get_approved(data,bckgrnd_start_input,bckgrund_stop_input,
                 ablation_start_input,ablation_stop_input,ablation_start_true,
                regression_buttons):
    data_toapprove = data.reset_index(drop=True)
    
    # need to send background subtracted data to regression, so do the following:
    background = data_toapprove[(data_toapprove.Time_s >= bckgrnd_start_input) & (data_toapprove.Time_s <= bckgrund_stop_input)]
    background = background.mean()[1:-1]
    background_subtracted_data = data_toapprove.iloc[:,2:-1].sub(background,axis='columns').clip(lower=0)
    data_toapprove.iloc[:,2:-1] = background_subtracted_data
    
    data_approved_ratio = data_toapprove.copy()
    data_approved_ratio = get_ratios(data_approved_ratio)
    data_approved_ratio = data_approved_ratio[(data_approved_ratio.Time_s >= ablation_start_input) & (data_approved_ratio.Time_s <= ablation_stop_input)]
    data_ratios,data_stats = get_approved_regressions(data_approved_ratio,regression_buttons,ablation_start_true)
    data_ratios = pd.DataFrame([data_ratios],columns=['206/238 1st Order','206/238 2nd Order','206Pb/204Pb','207Pb/206Pb','235U/238U'])
    # data_stats = pd.DataFrame([data_stats],columns=['R2 1st Order','R2 2nd Order','R2 Adj 2nd Order','SE 206/238 1st Order','SE% 206/238 1st Order','SE 206/238 2nd Order','SE% 206/238 2nd Order'])
    data_stats = pd.DataFrame([data_stats],columns=['R2 1st Order','R2 2nd Order','R2 Adj 2nd Order','SE 206/238 1st Order','SE% 206/238 1st Order','SE 206/238 2nd Order','SE% 206/238 2nd Order',
                                                   'SE 206/204','SE% 206/204','SE 207/206','SE% 207/206','SE 235/238','SE% 235/238'])
    data_toapprove_SE = pd.DataFrame([data_toapprove.iloc[:,2:-1].sem()]).add_suffix('_1SE')
    data_toapprove = pd.DataFrame([data_toapprove.iloc[:,2:-1].mean()])
    data_toapprove.insert(0,'SampleLabel',data.iloc[0,0])
    
    data_approved = data_toapprove.join([data_toapprove_SE,data_ratios,data_stats])
    
    return data_approved


def get_approved_regressions(data,regression_buttons,ablation_start_true):        
    if '1st Order' in regression_buttons:
        # y1, X1 = dmatrices('Pb206_U238c ~ Time_s', data=data, return_type='dataframe')
        y1, X1 = dmatrices('Pb206_U238 ~ Time_s', data=data, return_type='dataframe')
        mod1 = sm.OLS(y1, X1)
        fit1 = mod1.fit()
        predicted1 = fit1.params[0] + fit1.params[1]*data.Time_s
        rsquared1 = fit1.rsquared
        predicted_b01 = fit1.params[0] + fit1.params[1]*ablation_start_true
        sigma1 = np.sqrt(fit1.ssr/fit1.df_resid)
        # this isthe actual standard error for a point estiamted along a regression line
        # = sigma_regression * sqrt[1/n + (x_predicted - x_mean)^2 / ((df)*V(x))]
        # where sigma_regression is calculated above, x_predicted is the value at which you want to predict, x_mean is hte mean of all x
        # df is the degrees of freedom (n-#params), and V(x) is the variance of the predicted values
        SE_b01 = sigma1*np.sqrt(1/len(data)+(ablation_start_true-data['Time_s'].mean())**2/((len(data)-1)*statistics.variance(data['Time_s'])))
        SE_b01_percent = SE_b01/predicted_b01*100
    else:
        predicted1 = np.zeros_like(data['Time_s'])
        rsquared1 = 'NA'
        predicted_b01 = 'NA'
        SE_b01 = 'NA'
        SE_b01_percent = 'NA'
    
    if '2nd Order' in regression_buttons:
        # y2, X2 = dmatrices('Pb206_U238c ~ Time_s + Time_s2', data=data, return_type='dataframe')
        y2, X2 = dmatrices('Pb206_U238 ~ Time_s + Time_s2', data=data, return_type='dataframe')
        mod2 = sm.OLS(y2, X2)
        fit2 = mod2.fit()
        predicted2 = fit2.params[0] + fit2.params[1]*data.Time_s + fit2.params[2]*data.Time_s2
        rsquared2 = fit2.rsquared
        rsquared2_adj = fit2.rsquared_adj
        predicted_b02 = fit2.params[0] + fit2.params[1]*ablation_start_true + fit2.params[2]*ablation_start_true
        sigma2 = np.sqrt(fit2.ssr/fit2.df_resid)
        SE_b02 = sigma2*np.sqrt(1/len(data)+(ablation_start_true-data['Time_s'].mean())**2/((len(data)-1)*statistics.variance(data['Time_s'])))
        SE_b02_percent = SE_b02/predicted_b02*100
    else:
        predicted2 = np.zeros_like(data['Time_s'])
        rsquared2 = 'NA'
        rsquared2_adj = 'NA'
        predicted_b02 = 'NA'
        SE_b02 = 'NA'
        SE_b02_percent = 'NA'
        
    
    pb206_204 = data['Pb206_Pb204'].median()
    pb207_pb206 = data['Pb207_Pb206'].median()
    u235_u238 = data['U235_U238'].median()
    
    pb206_204SE = data['Pb206_Pb204'].sem()
    pb206_204SE_percent = data['Pb206_Pb204'].sem()/data['Pb206_Pb204'].median()
    pb207_206SE = data['Pb207_Pb206'].sem()
    pb207_206SE_percent = data['Pb207_Pb206'].sem()/data['Pb207_Pb206'].median()
    U235_U238SE = data['U235_U238'].sem()
    U235_U238SE_percent = data['U235_U238'].sem()/data['U235_U238'].median()
    
    ratios_to_return = [predicted_b01,predicted_b02,pb206_204,pb207_pb206,u235_u238]
    stats_to_return = [rsquared1,rsquared2,rsquared2_adj,SE_b01,SE_b01_percent,SE_b02,SE_b02_percent,pb206_204SE,pb206_204SE_percent,pb207_206SE,pb207_206SE_percent,U235_U238SE,U235_U238SE_percent]
    
    return ratios_to_return,stats_to_return

In [5]:
data_try = get_approved(data_input,3,28,33,50,32,'2nd Order')

/var/folders/h0/nxt863516yb3cyz22ybjvvm80000gn/T/ipykernel_39861/2856919065.py:73: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  background = background.mean()[1:-1]


In [6]:
# function block #

def get_ratios(data):
    og_len = len(data.columns)
    data = data.reset_index(drop=True)
    data_ratio = data.copy()
    pb206_pb204 = []
    pb206_u238 = []
    pb207_u235 = []
    pb207_pb206 = []
    u235_u238 = []
    pb206_u238c = []
    
    data_ratio['238c'] = data_ratio['235U'] * 137.82
    data_ratio['Time_s2'] = data_ratio['Time_s']**2
    
    for i in range(0,len(data_ratio)):
        if data_ratio['238U'][i] > 0:
            pb206_u238.append((data_ratio['206Pb'][i]/data_ratio['238U'][i]))
        else:
            pb206_u238.append(0)
    
    for i in range(0,len(data_ratio)):
        if data_ratio['204Pb'][i] > 0:
            pb206_pb204.append((data_ratio['206Pb'][i]/data_ratio['204Pb'][i]))
        else:
            pb206_pb204.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['235U'][i] > 0:
            pb207_u235.append((data_ratio['207Pb'][i]/data_ratio['235U'][i]))
        else:
            pb207_u235.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['206Pb'][i] > 0:
            pb207_pb206.append((data_ratio['207Pb'][i]/data_ratio['206Pb'][i]))
        else:
            pb207_pb206.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['238U'][i] > 0:
            u235_u238.append((data_ratio['235U'][i]/data_ratio['238U'][i]))
        else:
            u235_u238.append(0)
        
    for i in range(0,len(data_ratio)):
        if data_ratio['238c'][i] > 0:
            pb206_u238c.append((data_ratio['206Pb'][i]/data_ratio['238c'][i]))
        else:
            pb206_u238c.append(0)
        
    data_ratio['Pb206_Pb204'] = pb206_pb204
    data_ratio['Pb206_U238'] = pb206_u238
    data_ratio['Pb207_U235'] = pb207_u235
    data_ratio['Pb207_Pb206'] = pb207_pb206
    data_ratio['U235_U238'] = u235_u238
    data_ratio['Pb206_U238c'] = pb206_u238c
    
    data_ratio = data_ratio.iloc[:,(og_len-1):]
    
    return data_ratio



def get_approved(data,bckgrnd_start_input,bckgrund_stop_input,
                 ablation_start_input,ablation_stop_input,ablation_start_true,
                regression_buttons):
    data_toapprove = data.reset_index(drop=True)
    
    # need to send background subtracted data to regression, so do the following:
    background = data_toapprove[(data_toapprove.Time_s >= bckgrnd_start_input) & (data_toapprove.Time_s <= bckgrund_stop_input)]
    background = background.mean()[1:-1]
    background_subtracted_data = data_toapprove.iloc[:,2:-1].sub(background,axis='columns').clip(lower=0)
    data_toapprove.iloc[:,2:-1] = background_subtracted_data
    
    data_approved_ratio = data_toapprove.copy()
    data_approved_ratio = get_ratios(data_approved_ratio)
    data_approved_ratio = data_approved_ratio[(data_approved_ratio.Time_s >= ablation_start_input) & (data_approved_ratio.Time_s <= ablation_stop_input)]
    data_ratios,data_stats = get_approved_regressions(data_approved_ratio,regression_buttons,ablation_start_true)
    data_ratios = pd.DataFrame([data_ratios],columns=['206/238 1st Order','206/238 2nd Order','206Pb/204Pb','207Pb/206Pb','235U/238U'])
    # data_stats = pd.DataFrame([data_stats],columns=['R2 1st Order','R2 2nd Order','R2 Adj 2nd Order','SE 206/238 1st Order','SE% 206/238 1st Order','SE 206/238 2nd Order','SE% 206/238 2nd Order'])
    data_stats = pd.DataFrame([data_stats],columns=['R2 1st Order','R2 2nd Order','R2 Adj 2nd Order','SE 206/238 1st Order','SE% 206/238 1st Order','SE 206/238 2nd Order','SE% 206/238 2nd Order',
                                                   'SE 206/204','SE% 206/204','SE 207/206','SE% 207/206','SE 235/238','SE% 235/238'])
    data_toapprove_SE = pd.DataFrame([data_toapprove.iloc[:,2:-1].sem()]).add_suffix('_1SE')
    data_toapprove = pd.DataFrame([data_toapprove.iloc[:,2:-1].mean()])
    data_toapprove.insert(0,'SampleLabel',data.iloc[0,0])
    
    data_approved = data_toapprove.join([data_toapprove_SE,data_ratios,data_stats])
    
    return data_approved


def get_approved_regressions(data,regression_buttons,ablation_start_true):        
    if '1st Order' in regression_buttons:
        # y1, X1 = dmatrices('Pb206_U238c ~ Time_s', data=data, return_type='dataframe')
        y1, X1 = dmatrices('Pb206_U238 ~ Time_s', data=data, return_type='dataframe')
        mod1 = sm.OLS(y1, X1)
        fit1 = mod1.fit()
        predicted1 = fit1.params[0] + fit1.params[1]*data.Time_s
        rsquared1 = fit1.rsquared
        predicted_b01 = fit1.params[0] + fit1.params[1]*ablation_start_true
        sigma1 = np.sqrt(fit1.ssr/fit1.df_resid)
        # this isthe actual standard error for a point estiamted along a regression line
        # = sigma_regression * sqrt[1/n + (x_predicted - x_mean)^2 / ((df)*V(x))]
        # where sigma_regression is calculated above, x_predicted is the value at which you want to predict, x_mean is hte mean of all x
        # df is the degrees of freedom (n-#params), and V(x) is the variance of the predicted values
        SE_b01 = sigma1*np.sqrt(1/len(data)+(ablation_start_true-data['Time_s'].mean())**2/((len(data)-1)*statistics.variance(data['Time_s'])))
        SE_b01_percent = SE_b01/predicted_b01*100
    else:
        predicted1 = np.zeros_like(data['Time_s'])
        rsquared1 = 'NA'
        predicted_b01 = 'NA'
        SE_b01 = 'NA'
        SE_b01_percent = 'NA'
    
    if '2nd Order' in regression_buttons:
        # y2, X2 = dmatrices('Pb206_U238c ~ Time_s + Time_s2', data=data, return_type='dataframe')
        y2, X2 = dmatrices('Pb206_U238 ~ Time_s + Time_s2', data=data, return_type='dataframe')
        mod2 = sm.OLS(y2, X2)
        fit2 = mod2.fit()
        predicted2 = fit2.params[0] + fit2.params[1]*data.Time_s + fit2.params[2]*data.Time_s2
        rsquared2 = fit2.rsquared
        rsquared2_adj = fit2.rsquared_adj
        predicted_b02 = fit2.params[0] + fit2.params[1]*ablation_start_true + fit2.params[2]*ablation_start_true
        sigma2 = np.sqrt(fit2.ssr/fit2.df_resid)
        SE_b02 = sigma2*np.sqrt(1/len(data)+(ablation_start_true-data['Time_s'].mean())**2/((len(data)-1)*statistics.variance(data['Time_s'])))
        SE_b02_percent = SE_b02/predicted_b02*100
    else:
        predicted2 = np.zeros_like(data['Time_s'])
        rsquared2 = 'NA'
        rsquared2_adj = 'NA'
        predicted_b02 = 'NA'
        SE_b02 = 'NA'
        SE_b02_percent = 'NA'
        
    
    pb206_204 = data['Pb206_Pb204'].mean()
    pb207_pb206 = data['Pb207_Pb206'].mean()
    u235_u238 = data['U235_U238'].mean()
    
    pb206_204SE = data['Pb206_Pb204'].sem()
    pb206_204SE_percent = data['Pb206_Pb204'].sem()/data['Pb206_Pb204'].mean()*100
    pb207_206SE = data['Pb207_Pb206'].sem()
    pb207_206SE_percent = data['Pb207_Pb206'].sem()/data['Pb207_Pb206'].mean()*100
    U235_U238SE = data['U235_U238'].sem()
    U235_U238SE_percent = data['U235_U238'].sem()/data['U235_U238'].mean()*100
    
    ratios_to_return = [predicted_b01,predicted_b02,pb206_204,pb207_pb206,u235_u238]
    stats_to_return = [rsquared1,rsquared2,rsquared2_adj,SE_b01,SE_b01_percent,SE_b02,SE_b02_percent,pb206_204SE,pb206_204SE_percent,pb207_206SE,pb207_206SE_percent,U235_U238SE,U235_U238SE_percent]
    
    return ratios_to_return,stats_to_return




def get_residuals(data,regression_buttons):        
    if '1st Order' in regression_buttons:
        # y1, X1 = dmatrices('Pb206_U238c ~ Time_s', data=data, return_type='dataframe')
        y1, X1 = dmatrices('Pb206_U238 ~ Time_s', data=data, return_type='dataframe')
        mod1 = sm.OLS(y1, X1)
        fit1 = mod1.fit()
        predicted1 = fit1.params[0] + fit1.params[1]*data.Time_s
        resid1 = fit1.resid
        name1 = '1st Order'
    else:
        predicted1 = 0
        resid1 = 0
        name1 = 'NA'
    
    if '2nd Order' in regression_buttons:
        # y2, X2 = dmatrices('Pb206_U238c ~ Time_s + Time_s2', data=data, return_type='dataframe')
        y2, X2 = dmatrices('Pb206_U238 ~ Time_s + Time_s2', data=data, return_type='dataframe')
        mod2 = sm.OLS(y2, X2)
        fit2 = mod2.fit()
        predicted2 = fit2.params[0] + fit2.params[1]*data.Time_s + fit2.params[2]*data.Time_s2
        resid2 = fit2.resid
        name2 = '2nd Order'
    else:
        predicted2 = 0
        resid2 = 0
        name2 = 'NA'
        
    predicted_to_return = [predicted1,predicted2]
    resid_to_return = [resid1,resid2]
    names = [name1,name2]
    
    return predicted_to_return,resid_to_return,names



def get_regressions(data,regression_buttons,ablation_start_true):        
    if '1st Order' in regression_buttons:
        # y1, X1 = dmatrices('Pb206_U238c ~ Time_s', data=data, return_type='dataframe')
        y1, X1 = dmatrices('Pb206_U238 ~ Time_s', data=data, return_type='dataframe')
        mod1 = sm.OLS(y1, X1)
        fit1 = mod1.fit()
        predicted1 = fit1.params[0] + fit1.params[1]*data.Time_s
        rsquared1 = fit1.rsquared
        sigma1 = np.sqrt(fit1.ssr/fit1.df_resid)
        predicted_b01 = fit1.params[0] + fit1.params[1]*ablation_start_true
        SE_b01 = sigma1*np.sqrt(1/len(data)+(ablation_start_true-data['Time_s'].mean())**2/((len(data)-1)*statistics.variance(data['Time_s'])))
        SE_b01_percent = SE_b01/predicted_b01*100
    else:
        predicted1 = np.zeros_like(data['Time_s'])
        rsquared1 = 0
        predicted_b01 = 0
        SE_b01 = 0
        SE_b01_percent = 0
    
    if '2nd Order' in regression_buttons:
        # y2, X2 = dmatrices('Pb206_U238c ~ Time_s + Time_s2', data=data, return_type='dataframe')
        y2, X2 = dmatrices('Pb206_U238 ~ Time_s + Time_s2', data=data, return_type='dataframe')
        mod2 = sm.OLS(y2, X2)
        fit2 = mod2.fit()
        predicted2 = fit2.params[0] + fit2.params[1]*data.Time_s + fit2.params[2]*data.Time_s2
        rsquared2 = fit2.rsquared
        rsquared2_adj = fit2.rsquared_adj
        sigma2 = np.sqrt(fit2.ssr/fit2.df_resid)
        predicted_b02 = fit2.params[0] + fit2.params[1]*ablation_start_true + fit2.params[2]*ablation_start_true
        SE_b02 = sigma2*np.sqrt(1/len(data)+(ablation_start_true-data['Time_s'].mean())**2/((len(data)-1)*statistics.variance(data['Time_s'])))
        SE_b02_percent = SE_b02/predicted_b02*100
    else:
        predicted2 = np.zeros_like(data['Time_s'])
        rsquared2 = 0
        rsquared2_adj = 0
        predicted_b02 = 0
        SE_b02 = 0
        SE_b02_percent = 0
    
    regressions_to_return = [predicted1,predicted2]
    stats_to_report = [rsquared1,rsquared2,rsquared2_adj,SE_b01_percent,SE_b02_percent]
    # intercepts_to_return = []
    
    return regressions_to_return,stats_to_report



def ablation_plot(data_ablation_,bckgrnd_start_input,bckgrund_stop_input,
                  ablation_start_input,ablation_stop_input,ablation_start_true,ablation_plot_ylim_slider):
    """
    """
    if data_ablation_ is None:
        return 'There is something wrong with the plot input'
    else:
        plt.style.use('seaborn-colorblind')
        fig = Figure(figsize=(12,8))
        ax = fig.add_subplot()
        var_cols = data_ablation_.columns[3:-1]
        # data_ablation_['Time_s'] = data_ablation_['Time']/1000
        for i in var_cols:
            ax.plot(data_ablation_.Time_s,data_ablation_[i],'-',lw=1,label='{}'.format(i))
        ax.plot([bckgrnd_start_input,bckgrnd_start_input],[0,1e8],'--k',lw=0.75)
        ax.plot([bckgrund_stop_input,bckgrund_stop_input],[0,1e8],'--k',lw=0.75)
        ax.plot([ablation_start_input,ablation_start_input],[0,1e8],'-k',lw=0.75)
        ax.plot([ablation_stop_input,ablation_stop_input],[0,1e8],'-k',lw=0.75)
        ax.plot([ablation_start_true,ablation_start_true],[0,1e8],':',c='k',lw=0.75)
        ax.legend(loc='upper right',fontsize=18)
        ax.set_ylim(0,ablation_plot_ylim_slider)
        ax.set_xlabel('Time (s)',fontsize=18)
        ax.set_ylabel('Counts',fontsize=18)
        ax.tick_params(axis='both',labelsize=14)
        fig.tight_layout()
        return fig

    
def ratio_plot(data,ratio_buttons,regression_buttons,
               start_bckgrnd,stop_bckgrnd,
               start_ablation,stop_ablation,ablation_start_true,
               # ratio_plot_ylim_slider):
               ratio_plot_ylim_slider_min,ratio_plot_ylim_slider_max):
    """
    """
    plt.style.use('seaborn-colorblind')
    fig = Figure(figsize=(12,8))
    ax = fig.add_subplot()
    data = get_ratios(data)
    data_to_regress = data[(data.Time_s >= start_ablation) & (data.Time_s <= stop_ablation)]
    var_cols = ratio_buttons
    regressions_to_plot = regression_buttons
    regressions,stats = get_regressions(data_to_regress,regression_buttons,ablation_start_true)
    
    for i in var_cols:
        ax.plot(data.Time_s,data[i],'-',lw=1,label='{}'.format(i))
        
    if regressions_to_plot is not None:
        for i in regressions:
            ax.plot(data_to_regress.Time_s,i)
    
    ax.plot([start_bckgrnd,start_bckgrnd],[0,max(data['Pb206_U238'])],'--k',lw=0.75)
    ax.plot([stop_bckgrnd,stop_bckgrnd],[0,max(data['Pb206_U238'])],'--k',lw=0.75)
    ax.plot([start_ablation,start_ablation],[0,max(data['Pb206_U238'])],'-k',lw=0.75)
    ax.plot([stop_ablation,stop_ablation],[0,max(data['Pb206_U238'])],'-k',lw=0.75)
    ax.plot([ablation_start_true,ablation_start_true],[0,1e8],':',c='k',lw=0.75)
    ax.legend(loc='upper right',fontsize=18)
    ax.set_ylim(ratio_plot_ylim_slider_min,ratio_plot_ylim_slider_max)
    # ax.set_ylim(0,ratio_plot_ylim_slider)
    ax.set_xlabel('Time (s)',fontsize=18)
    ax.set_ylabel('Selected Ratios',fontsize=18)
    ax.tick_params(axis='both',labelsize=14)
    fig.tight_layout()
    # stats_to_report = [pears_chi_1,pears_chi_2,rsquared1,rsquared2,rsquared2_adj,LRT_p]
    return fig



def residuals_plot(data,regression_buttons,start_ablation,stop_ablation):
    """
    """
    plt.style.use('seaborn-colorblind')
    fig = Figure(figsize=(6,3))
    ax = fig.add_subplot()
    data = get_ratios(data)
    data_to_regress = data[(data.Time_s >= start_ablation) & (data.Time_s <= stop_ablation)]
    fitted,residuals,names = get_residuals(data_to_regress,regression_buttons)
    
    for i,k,j in zip(fitted,residuals,names):
        if i is not 0:
            ax.plot(i,k,'o',mec='k',label='{}'.format(j))
            ax.plot([min(i),max(i)],[0,0],'-k',lw=0.5)
        else:
            pass
    
    ax.legend(loc='upper right',fontsize=8)
    ax.set_xlabel('Fitted Value',fontsize=10)
    ax.set_ylabel('Residuals',fontsize=10)
    ax.tick_params(axis='both',labelsize=8)
    fig.tight_layout()
    return fig


<>:318: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:318: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/h0/nxt863516yb3cyz22ybjvvm80000gn/T/ipykernel_39861/3663482645.py:318: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if i is not 0:


In [7]:
class make_plots(param.Parameterized):
    
    # set up parameters. those with the suffix _pass are set up in order to be updated in the first callback below.
    input_data = param.DataFrame(default=data_input,precedence=-1)
    blank_output = pd.DataFrame([np.zeros(len(data_input.columns))],columns=list(data_input.columns))
    output_data = param.DataFrame(default=blank_output,precedence=-0.2)
    ablation_start_true = param.Integer(30)
    ablation_slider = param.Range(default=(33,55),bounds=(0,100))
    background_slider = param.Range(default=(3,28),bounds=(0,100))
    ablation_plot_ylim_slider = param.Integer(default=1000000,bounds=(0,10000000))
    # ratio_plot_ylim_slider = param.Number(default=1,bounds=(0,5))
    ratio_plot_ylim_min = param.Number(default=0.01)
    ratio_plot_ylim_max = param.Number(default=0.5)
    # ratio_plot_ylim_slider = param.Range(default=(0.01, 0.05), bounds=(0.01, 1.05))
    sample_subset = param.Selector(objects=options)
    # update_output_button = pn.widgets.Button(name='Approve Subset',button_type='primary')
    update_output_button = param.Action(lambda x: x.add_output_data(),label='Approve Interval')
    export_data_button = param.Action(lambda x: x.export_data(),label='DDDT!')
    ratio_buttons = param.ListSelector(default=['Pb206_U238'], objects=['Pb206_U238','Pb206_Pb204','Pb204_U235','Pb207_Pb206','U235_U238','Pb206_U238c'])
    regression_buttons = param.ListSelector(default=['1st Order'], objects=['1st Order','2nd Order'])
    
    def __init__(self,**params):
        super().__init__(**params)
        self.output_data_widget = pn.Param(self.param.output_data),
        self.input_data_widget = pn.Param(self.param.input_data),
        self._layout = pn.Row(
            pn.Param(self,parameters=['update_output_button','sample_subset','ablation_slider','background_slider',
                                      'ablation_plot_ylim_slider','log_y_ablation',
                                      'ratio_plot_ylim_min','ratio_plot_ylim_max'
                                      # 'ratio_plot_ylim_slider',
                                      ]
                    ),
            pn.Param(widgets={'ratio_buttons': pn.widgets.CheckBoxGroup, 'regression_buttons': pn.widgets.CheckBoxGroup,
                             'export_data_button': pn.widgets.CheckBoxGroup(name='DDDT',button_type='success')}),
            self.input_data_widget,
            self.output_data_widget,
            width=1,
            sizing_mode='fixed'
        )
        
    @pn.depends('output_data',watch=True)
    def _update_output_widget(self):
        self.output_data_widget = self.output_data
        self.output_data_widget.height = 40
        self.output_data_widget.heightpolicy = 'Fixed'
        return pn.widgets.Tabulator(self.output_data_widget,width=100)
        
    @pn.depends('input_data','background_slider','ablation_slider','ablation_start_true',
                'sample_subset','ablation_plot_ylim_slider')
    def call_ablation_plot(self):
        if self.input_data is not None:
            data_toplot = self.input_data[self.input_data['SampleLabel'] == self.sample_subset]
            return ablation_plot(data_toplot,self.background_slider[0],self.background_slider[1],
                                 self.ablation_slider[0],self.ablation_slider[1],self.ablation_start_true,self.ablation_plot_ylim_slider)
    
    @pn.depends('input_data','ratio_buttons','regression_buttons',
                'background_slider','ablation_slider','ablation_start_true',
                'sample_subset',
                'ratio_plot_ylim_min','ratio_plot_ylim_max')
                # 'ratio_plot_ylim_slider')
    def call_ratio_plot(self):
        data_toplot = self.input_data[self.input_data['SampleLabel'] == self.sample_subset]
        return ratio_plot(data_toplot,self.ratio_buttons,self.regression_buttons,self.background_slider[0],self.background_slider[1],
                          self.ablation_slider[0],self.ablation_slider[1],self.ablation_start_true,
                          # self.ratio_plot_ylim_slider)
                          # self.ratio_plot_ylim_slider[0],self.ratio_plot_ylim_slider[1])
                          self.ratio_plot_ylim_min,self.ratio_plot_ylim_max)
    
    
    @pn.depends('input_data','regression_buttons','ablation_slider','sample_subset')
    def call_residuals_plot(self):
        if self.input_data is not None:
            data_toplot = self.input_data[self.input_data['SampleLabel'] == self.sample_subset]
            return residuals_plot(data_toplot,self.regression_buttons,self.ablation_slider[0],self.ablation_slider[1])
        
        
    @pn.depends('input_data','regression_buttons','ablation_slider','sample_subset','ablation_start_true')
    def get_regression_stats(self):
        if self.input_data is not None:
            data_toanalyze = self.input_data[self.input_data['SampleLabel'] == self.sample_subset]
            data_toanalyze = get_ratios(data_toanalyze)
            data_toanalyze = data_toanalyze[(data_toanalyze.Time_s >= self.ablation_slider[0]) & (data_toanalyze.Time_s <= self.ablation_slider[1])]
            regressions,stats = get_regressions(data_toanalyze,self.regression_buttons,self.ablation_start_true)
            r2_1 = pn.pane.Markdown('$$R^{2}_{1st}$$ = '+(str(round(stats[0],3))))
            r2_2 = pn.pane.Markdown('$$R^{2}_{2nd}$$ = '+(str(round(stats[1],3))))
            r2adj_2 = pn.pane.Markdown('$$R^{2adj.}_{2nd}$$ = '+(str(round(stats[2],3))))
            SE_1stper = pn.pane.Markdown('$$1^{st} Order SE \%$$ = '+(str(round(stats[3],3))))
            SE_2ndper = pn.pane.Markdown('$$2^{nd} Order SE \%$$ = '+(str(round(stats[4],3))))
            stats_markdown = pn.Column(r2_1,r2_2,r2adj_2,SE_1stper,SE_2ndper)
            return stats_markdown
        
    

    def __panel__(self):
        return self._layout
    
    
    def add_output_data(self, event=None):
        data_tosend = self.input_data[self.input_data['SampleLabel'] == self.sample_subset]
        data_approved = get_approved(data_tosend,self.background_slider[0],self.background_slider[1],self.ablation_slider[0],self.ablation_slider[1],
                                     self.ablation_start_true,self.regression_buttons)
        # data_tosend = data_tosend[(data_tosend.Time_s >= self.ablation_start_slider) & (data_tosend.Time_s <= self.ablation_stop_slider)]
        if self.output_data is None:
            self.output_data = data_approved
        else:
            self.output_data = self.output_data.append(data_approved,ignore_index=True)
    
    @pn.depends('output_data')
    def export_data(self,event=None):
        self.output_data.to_excel('output_lasertramZ.xlsx')
        
    
ablation_ = make_plots(name='Reduce U-Pb Data!')


In [8]:
pn.extension()

# fixed, stretch_width, stretch_height, stretch_both, scale_width, scale_height, scale_both, None]
width_ratios=[5,5,2]
grid_layout = pn.GridSpec(sizing_mode='scale_both',width_ratios=width_ratios)

grid_layout[0,0] = ablation_.call_ablation_plot
grid_layout[0,1] = ablation_.call_ratio_plot

grid_layout[1,0] = ablation_.call_residuals_plot
grid_layout[1,1] = ablation_.get_regression_stats

grid_layout[:,2] = pn.WidgetBox(pn.Param(ablation_.param,
                                         widgets={'regression_buttons': pn.widgets.CheckBoxGroup,
                                                  'ratio_buttons': pn.widgets.CheckBoxGroup,
                                                  'export_data_button': pn.widgets.Button(name='DDDT!',button_type='success')}
                                        )
                               )
# grid_layout[:,2] = pn.Column(pn.WidgetBox
#                              (pn.Param(ablation_.param,
#                                          widgets={'regression_buttons': pn.widgets.CheckBoxGroup,
#                                                   'ratio_buttons': pn.widgets.CheckBoxGroup,
#                                                   'export_data_button': pn.widgets.Button(name='DDDT!',button_type='success')}
#                                       )
#                                ),
#                              width=2,
#                              sizing_mode='fixed'
#                             )
grid_layout[2,0] = ablation_._update_output_widget


grid_layout.show()

Launching server at http://localhost:57188
